In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle


In [3]:
! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [4]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
! kaggle datasets download vipoooool/new-plant-diseases-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [6]:
!unzip new*

unzip:  cannot find or open new*, new*.zip or new*.ZIP.

No zipfiles found.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [10]:
len(os.listdir("/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"))

FileNotFoundError: ignored

In [11]:
train_datagen = ImageDataGenerator(zoom_range=0.5, shear_range=0.3, horizontal_flip=True, preprocessing_function=preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [12]:
train = train_datagen.flow_from_directory(directory="/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train", target_size=(256,256), batch_size=32)

val = val_datagen.flow_from_directory(directory="/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid", target_size=(256,256), batch_size=32)

FileNotFoundError: ignored

MODEL BUILDING

In [13]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [14]:
base_model = VGG19(input_shape=(256, 256, 3), include_top=False)

80150528/80134624 [==============================] - 2s 0us/step


In [15]:
for layer in base_model.layers:
  layer.trainable=False

In [16]:
X = Flatten()(base_model.output)

X = Dense(units=38, activation='softmax')(X)

model = Model(base_model.input, X)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [18]:
model.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [19]:
#early stopping and model checkpoint

from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 3, verbose = 1)

#model checkpoint
mc = ModelCheckpoint(filepath = "best_model.h5", min_delta = 0.01, patience = 3, verbose = 1, save_best_onlu = True)

cb = [es, mc]

In [20]:
his = model.fit_generator(train, steps_per_epoch=16, epochs = 50, verbose = 1, callbacks = cb, validation_data=val, validation_steps=16)

NameError: ignored

In [21]:
h = his.history
h.keys()

NameError: ignored

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = "red")
plt.title("acc vs val_acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = "red")
plt.title("loss vs val_loss")
plt.show()

In [ ]:
#load best model

from keras.models import load_model

model = load_model("/content/best_model.h5")

In [ ]:
acc = model.evaluate_generator(val)[1]

print(f"The accuracy of your model is = {acc*100} %")

In [ ]:
ref = dict(zip( list(train.class_indices.values()), list(train.class_indices.keys())) )

In [ ]:
def prediction(path):

  img = load_img(path, target_size = (256,256))

  i = img_to_array(img)

  im = preprocess_input(i)

  img = np.expand_dims(im, axis = 0)

  pred = np.argmax(model.predict(img))

  print(f" the image belongs to { ref[pred] } ")

In [ ]:
path = "/content/test/test/AppleScab1.JPG"

prediction(path)